## Easy
Чтобы найти медиану можно использовать partition для центрального элемента. Если колчиество элементов чётное, то берём среднее арифметическое от n//2 и n//2 - 1
\
Чтобы найти моду, после того, как прошли по списку, сохраняем наибольшую частоту из всех элементов, после этого идём по ссловарю и ищем все элементы с такой частотой. Берём среднее арифметическое от них

In [1]:
from numpy import partition

def func(arr):
    d = {}
    res = {}
    # new_arr = sorted(arr)
    l = len(arr)
    mean = 0
    median = 0
    mode = 0
    if(l == 0):
        res["mean"] = 0
        res["median"] = None
        res["mode"] = None
        return res
    arr1 = partition(arr, l//2)
    if(l % 2 == 1):
        median = arr1[l//2]
    else:
        arr2 = partition(arr1, l//2 - 1)
        median = (arr2[l//2] + arr2[l//2 - 1])/2

    max_key = None    
    for a in arr:
        mean += a
        d[a] = d.get(a, 0) + 1
        if(max_key == None):
            max_key = a
        elif(d[max_key] < d[a]):
            max_key = a
    
    mean /= l
    count = 0
    for k in d.keys():
        if(d[k] == d[max_key]):
            mode += k
            count += 1
    mode /= count
    res["mean"] = mean
    res["median"] = median
    res["mode"] = mode
    return res

## Test

In [2]:
example = [[0, 1, 1, 10, 5, 4, 3], [-6, 2, -6, -6, 8, -1, -1, 0, 0, 0], []]

for i in range(0, len(example)):
    print(func(example[i]))

{'mean': 3.4285714285714284, 'median': 3, 'mode': 1.0}
{'mean': -1.0, 'median': -0.5, 'mode': -3.0}
{'mean': 0, 'median': None, 'mode': None}


## Medium
Идём по строкам и сплитим их по разделяющему знаку. Каждый элемент в строке обрабатываем отдельно

In [3]:
import re

def parse_csv(file_content: str) -> list[dict]:
    result = list()
    flag = False
    att = list()
    for s in file_content:
        if(not flag):
            flag = True
            att = s.split(',')
        else:
            line = s.split(',', len(att)-1)
            d = {}
            for i in range(0, len(att)):
                if(att[i].strip() in ("", "id", "legs")):
                    d[att[i].strip()] = int(line[i])
                elif(att[i].strip() == 'roles'):
                    d[att[i].strip()] = re.findall(r"\w+", line[i]) 
                else:
                    d[att[i].strip()] = line[i]
            result.append(d)
    return result

def max_leg_hero(parsed_content: list[dict]) -> str:
    id = 0
    for i in range(0, len(parsed_content)):
        if(parsed_content[i]["legs"] > parsed_content[id]["legs"]):
            id = i
    return parsed_content[id]["localized_name"]

## Test

In [4]:
with open("../data/dota_hero_stats.csv") as f:
    d = parse_csv(f)
    # for l in d:
    #     print(l)
    #     print('\n') 
    print(max_leg_hero(d))

Broodmother


## Hard
![01-hw-img.png](../images/01-hw-img.png)
\
Возьмём метрику (x-attack != y-attack) + |x-legs - y-legs| + (x-attr != y-attr) + (|x-roles / y-roles| + |y-roles / x-roles|)
\
Это действительно метрика, т.к. аксиома положительности, тождества и симметричности сохраняются. Аксиома треугольника: возьмём ещё z, для модуля всё очевидно, посмотрим на выражения с != : (x-attack != z-attack + z-attack != y-attack) >= ((x-attack != z-attack) V (z-attack != y-attack)) = (x-attack != y-attack)
\
Скобка с attr так же оценивается
\
Скобка с roles оценивается:(|x-roles / z-roles| + |z-roles / x-roles|) + (|z-roles / y-roles| + |y-roles / z-roles|) \geq |x_{\text{roles}} / y_{\text{roles}}| + |y_{\text{roles}} / x_{\text{roles}}|\) - очевидно, т.к. добавляется множество \(z\)

In [5]:

def most_similar_heroes(data: list[dict]):
    result = (None, None, -1)
    for i in range(0, len(data)):
        for j in range(i+1, len(data)):
            diff_role = 0
            s1 = set(data[i]["roles"])
            s2 = set(data[j]["roles"])
            metric = ((data[i]["attack_type"] != data[j]["attack_type"]) + abs(data[i]["legs"] - data[j]["legs"]) + 
                          (data[i]["primary_attr"] != data[j]["primary_attr"]) + (len((s1-s2)) + len((s2-s1))))
            if(result[2] == -1 or result[2] > metric):
                result = (data[i]["localized_name"], data[j]["localized_name"], metric)
    return result

In [6]:
res = most_similar_heroes(d)
print(res)

('Crystal Maiden', 'Keeper of the Light', 0)
